# ConsensusClusterPlus in R

In [24]:
#library("wateRmelon")
# https://bioconductor.org/packages/release/bioc/vignettes/wateRmelon/inst/doc/wateRmelon.pdf
# https://bioconductor.riken.jp/packages/3.3/bioc/manuals/wateRmelon/man/wateRmelon.pdf
#library(methylumi)

# run consensus cluster plus: http://bioconductor.org/packages/release/bioc/vignettes/ConsensusClusterPlus/inst/doc/ConsensusClusterPlus.pdf
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")
#BiocManager::install("ConsensusClusterPlus")
library(ConsensusClusterPlus)

In [2]:
# TCGA data files have likely already undergone background correction adn dye-bias correction. 
# this should control for  typeI/typeII differences, but doesnt seem to entirely. 
# moot if using beta values with binary mask

starting httpd help server ...
 done



In [55]:
# load raw beta vals
top_prbs = read.csv("./df_tcga_top1K_v1_probes_crc.csv", header=TRUE)
rownames(top_prbs) <- top_prbs[,colnames(top_prbs)[1]]
top_prbs <- data.matrix(transpose(top_prbs[,2:length(top_prbs)]))

dim(top_prbs)
head(top_prbs)

[1]  361 1000

,cg03616722,cg18222083,cg11738485,cg17300544,cg17510056,cg06509223,cg13205384,cg20196215,cg20776829,cg12539796,⋯,cg04336836,cg19850348,cg14453201,cg12737152,cg23045719,cg13742526,cg23539933,cg09508531,cg02484469,cg09279240
TCGA-CI-6620-01A-11D-1828-05,0.2730491,0.02571368,0.6256951,0.04824680,0.04478184,0.91205278,0.33539434,0.5760381,0.02742225,0.04083147,⋯,0.58591482,0.45683237,0.52459655,0.02362268,0.38966678,0.29742691,0.58029465,0.03273436,0.418613655,0.03402600
TCGA-EI-6882-01A-11D-1926-05,0.4103747,0.61567129,0.6379284,0.04203827,0.04562850,0.89243691,0.94027397,0.9136101,0.56800211,0.22900816,⋯,0.34082954,0.53476553,0.90335168,0.60098457,0.33387984,0.74309022,0.63245789,0.17196879,0.826279514,0.40971143
TCGA-CI-6622-01A-11D-1828-05,0.1649976,0.02327784,0.9828958,0.08895300,0.69634445,0.07302612,0.79601241,0.1579945,0.74454390,0.02575274,⋯,0.04505240,0.07522610,0.70153285,0.54695307,0.01515970,0.07935585,0.11735846,0.02065233,0.768215891,0.02574433
TCGA-EI-6509-01A-11D-1734-05,0.1234680,0.02997401,0.5362339,0.70750864,0.40019566,0.54222743,0.04785236,0.2369834,0.01776330,0.02825046,⋯,0.02867564,0.01163012,0.03489097,0.01796650,0.01161259,0.01731528,0.05704351,0.01520891,0.009380666,0.01833218
TCGA-DY-A1H8-01A-21D-A153-05,0.9022145,0.93627649,0.9779110,0.04167064,0.05164847,0.92896820,0.08098451,0.7795124,0.01779990,0.03759749,⋯,0.32671495,0.02685311,0.67170719,0.03685075,0.01574805,0.77454629,0.07467122,0.02188353,0.063003452,0.01874133
TCGA-DC-5869-01A-01D-1658-05,0.2094590,0.22345307,0.5960873,0.08032846,0.92722371,0.34927217,0.09156665,0.2533636,0.57045543,0.03333091,⋯,0.27697004,0.01700004,0.56732877,0.48600957,0.57088402,0.28456823,0.08893853,0.28373321,0.747147159,0.02652155


In [78]:
# not going to normalize values here because all probes are on the same 0-1 scale, 
# should not cause one probe to have more influence than another. 
top_prbs <- t(top_prbs)

In [ ]:
#Consensus cluster masked data 

# load mask data
top_prbs_msk = read.csv("./df_tcga_top1k_v1_probes_crc_pt2mask.csv", header=TRUE)
rownames(top_prbs_msk) <- top_prbs_msk[,colnames(top_prbs_msk)[1]]
top_prbs_msk <- data.matrix(t(top_prbs_msk[,2:length(top_prbs_msk)]))

dim(top_prbs_msk)
#head(top_prbs_msk)

# Hierarchical clustering produces some clusters with only 1 sample 
#results_msk_hc = ConsensusClusterPlus(top_prbs_msk, maxK=8, reps=10, pItem=0.8, pFeature=1, title="./ConClust_hc_mask",
#                              clusterAlg="hc", distance="binary", seed=100, plot="png", writeTable=TRUE,
#                              verbose=FALSE)

results_msk_kmeans = ConsensusClusterPlus(top_prbs_msk, maxK=8, reps=10, pItem=0.8, pFeature=1, title="./ConClust_kmeans_mask",
                              clusterAlg="kmdist", distance="binary", seed=100, plot="png", writeTable=TRUE,
                              verbose=FALSE)

In [92]:
# cluster labels are automatically written to outfiles:
labels = read.csv("./ConClust_kmeans_mask/ConClust_kmeans_mask.k=2.consensusClass.csv", header=FALSE)
colnames(labels) <- c("sample","cluster")
head(labels)




,sample,cluster
,<fct>,<int>
1,TCGA-CI-6620-01A-11D-1828-05,1
2,TCGA-EI-6882-01A-11D-1926-05,1
3,TCGA-CI-6622-01A-11D-1828-05,1
4,TCGA-EI-6509-01A-11D-1734-05,2
5,TCGA-DY-A1H8-01A-21D-A153-05,2
6,TCGA-DC-5869-01A-01D-1658-05,1
